In [10]:
import torch
import pytorch_lightning as pl
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [8]:
class NN(pl.LightningDataModule):
    def __init__(self, input_size, ouptut_size) -> None:
        super().__init__()
    
    def forward(self, x):
        ...

    def training_step(self,  batch, batch_idx):
        x, y = batch
        scores = self.forward(x)
        loss = nn.CrossEntropyLoss(scores, y)
        self.log("training loss", loss)
        return loss
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [81]:
class mymodel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.F = F.tanh
        self.lin2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.lin2(self.F(self.lin1(x)))
        return out

In [12]:
train_set = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
test_set = torchvision.datasets.MNIST(
    root="./data", train=False, transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2601836.53it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 347967.09it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 1705685.97it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1515193.57it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [88]:
input_size = 28 * 28
hidden_size = 1000
output_size = 10
learning_rate = 0.01
batch_size = 200
num_epoch = 5

model = mymodel(input_size=input_size,
                hidden_size=hidden_size, output_size=output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loader = torch.utils.data.DataLoader(
    dataset=train_set, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(
    dataset=test_set, shuffle=False, batch_size=batch_size)

In [89]:
total_steps = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_size)

        outputs = model(images)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f"epoch {epoch + 1} / {num_epoch}, step {i + 1}/{total_steps}, loss = {loss.item()}")

with torch.no_grad():
    correct = 0
    num = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        num += labels.shape[0]
        correct += (predictions == labels).sum().item()
    print(f'acc = {100 * correct / num} %')

epoch 1 / 5, step 100/300, loss = 0.3075460195541382
epoch 1 / 5, step 200/300, loss = 0.2759743630886078
epoch 1 / 5, step 300/300, loss = 0.3815400302410126
epoch 2 / 5, step 100/300, loss = 0.33134832978248596
epoch 2 / 5, step 200/300, loss = 0.21337170898914337
epoch 2 / 5, step 300/300, loss = 0.1634812355041504
epoch 3 / 5, step 100/300, loss = 0.12714354693889618
epoch 3 / 5, step 200/300, loss = 0.09536603838205338
epoch 3 / 5, step 300/300, loss = 0.12542006373405457
epoch 4 / 5, step 100/300, loss = 0.1704832911491394
epoch 4 / 5, step 200/300, loss = 0.15308034420013428
epoch 4 / 5, step 300/300, loss = 0.07431080937385559
epoch 5 / 5, step 100/300, loss = 0.06686179339885712
epoch 5 / 5, step 200/300, loss = 0.036693230271339417
epoch 5 / 5, step 300/300, loss = 0.26770293712615967
acc = 95.66 %


In [39]:
from PIL import Image
import numpy as np

In [127]:
with torch.no_grad():
    test = Image.open("test.png")
    im = torch.tensor(np.array(test.convert("L")).reshape(
        1, 28*28), dtype=torch.float32)
    _, index = torch.max(model(im), 1)
    print(index.item())

3
